# Power Consumption House

## MLP Online vs MLP Batch


### MLP Online

### MLP Batch

### Confronto MLP Batch vs MLP Online


## KNN Regressor Batch vs Online


### KNN regressor Online


### Confronto KNN regressor Batch vs KNN regressor Online


## Random Forest Online vs Random Forest Batch


### Random Forest Online


### Random Forest Batch


### Confronto Random Forest Batch vs Random Forest Online


## Decision Tree Online vs Decision Tree Batch


### Decision Tree Online


### Decision Tree Batch


### Confronto Decision Tree Batch vs Decision Tree Online